In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import datetime
import pickle
import os.path
import base64
import pandas as pd
from io import BytesIO
import os
import openpyxl
import mysql.connector
from sshtunnel import SSHTunnelForwarder
import pymysql
import mysql.connector
from requests.structures import CaseInsensitiveDict
import requests

# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def get_gmail_service():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '/Users/germankosenkov/Code projects/Crawling/3. Crawling Emails/New/client_secret_811893502700-tj4c2bv2942q1ua3459au95sc1ohoqle.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    return service

def search_messages(service, query):
    result = service.users().messages().list(userId='me', q=query).execute()
    messages = []
    if 'messages' in result:
        messages.extend(result['messages'])
    return messages

def get_attachments(service, message_id, store_dir, desired_filename):
    try:
        message = service.users().messages().get(userId='me', id=message_id).execute()
        parts = message.get('payload', {}).get('parts', [])
        for part in parts:
            if part['filename'] == desired_filename:  # Check if the filename matches the desired one
                if 'data' in part['body']:
                    data = part['body']['data']
                else:
                    att_id = part['body'].get('attachmentId')
                    att = service.users().messages().attachments().get(userId='me', messageId=message_id, id=att_id).execute()
                    data = att['data']

                file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))
                path = os.path.join(store_dir, part['filename'])

                if not os.path.exists(store_dir):
                    os.makedirs(store_dir)
                with open(path, 'wb') as f:
                    f.write(file_data)
    except Exception as e:
        print(f"An error occurred: {e}")


current_date = datetime.date.today()
current_week = current_date.isocalendar()[1]
previous_week = current_week - 1

current_year = datetime.date.today().year

new_file_path = ''


def main():
    service = get_gmail_service()
    search_query = 'from:XXXX.org has:attachment filename:XXXX.xlsx'
    messages = search_messages(service, search_query)

    if messages:
        latest_message = messages[0]  # Get the latest email

        # Construct the path for saving the attachment with the week and year in the file name
        #folder_name = f'{current_year}_Week_{previous_week}'
        save_path = '/Users/germankosenkov/Code projects/Crawling/3. Crawling Emails/New/XXX_exports'
        

        # Call get_attachments with the desired filename
        get_attachments(service, latest_message['id'], save_path, 'XXXX.xlsx')

        # Path to the downloaded Excel file
        old_file_path = os.path.join(save_path, 'XXXX.xlsx')
        new_file_path = os.path.join(save_path, f'{current_year}_Week_{previous_week}.xlsx')

        try:
            # Rename the downloaded file
            os.rename(old_file_path, new_file_path)
            return new_file_path

            # excel_data = pd.read_excel(new_file_path)
            # print(excel_data)
        except Exception as e:
            print(f"Error reading or renaming the Excel file: {e}")

if __name__ == '__main__':
    new_file_path = main()
    print(f"New file path: {new_file_path}")


New file path: /Users/germankosenkov/Code projects/Crawling/3. Crawling Emails/New/RUS_exports/2024_Week_8.xlsx


In [2]:
import openpyxl
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import datetime
import pickle
import os.path
import base64
import pandas as pd
from io import BytesIO
import os
import openpyxl
import mysql.connector
from sshtunnel import SSHTunnelForwarder
import pymysql
import mysql.connector
from requests.structures import CaseInsensitiveDict
import requests
from Constants import products_matched, dates_22_23, dates_23_24

# Load the Excel workbook
workbook = openpyxl.load_workbook(new_file_path, data_only=True)

# Select the sheet you want to extract data from
sheet = workbook['OIL']


column_names = []


b_ssh_host = 'XXXX'
b_ssh_user = 'XXXX'
b_ssh_port = 'XXXX'
b_ssh_private_key = 'XXXX'
b_sql_hostname = 'XXXX'
b_sql_username = 'XXXX'
b_sql_password = 'XXXX'
b_sql_database = 'XXXX'
b_sql_port = 'XXXX'

with SSHTunnelForwarder(
        (b_ssh_host, b_ssh_port),
        ssh_username=b_ssh_user,
        ssh_pkey=b_ssh_private_key,
        remote_bind_address=(b_sql_hostname, b_sql_port)) as tunnel:

    b_conn = mysql.connector.connect(
        host='127.0.0.1',
        user=b_sql_username,
        passwd=b_sql_password,
        db=b_sql_database,
        port=tunnel.local_bind_port
    )

    b_cursor = b_conn.cursor()

    # Iterate through headers to get column names
    for headers in sheet.iter_rows(min_row=3, max_row=3, min_col=3):
        for cell in headers:
            header_value = cell.value
            if header_value is not None and len(str(header_value)) < 4:
                column_names.append(header_value)

    cell_value_above = None

    # Iterate through rows starting from row 4 (assuming data starts from row 4)
    for row_num in range(4, 15, 2):  # Increment by 2 to process two rows at a time
        product_name_in_sheet = sheet.cell(row=row_num, column=1).value  # Get the product name in the current row



        check_data_query = "SELECT * FROM TradeData WHERE product_id = %s AND date = %s AND trade_quantity = %s AND origin_country_id = %s AND destination_country_id_raw = %s AND data_source_id = %s"
        
        if product_name_in_sheet is None:
            product_name = cell_value_above
        else:
            product_name = product_name_in_sheet

        # Extract product ID based on product name
        product_id = products_matched.get(product_name, None)

        if product_id is None:
            continue

        corrected_date = None

        def correct_date_1(header_value):
            return dates_23_24.get(header_value, None)

        def correct_date_2(header_value):
            return dates_22_23.get(header_value, None)

        # Initialize dictionaries to store data values for selected columns in two rows
        data_values_row1 = {}
        data_values_row2 = {}

        # Iterate through columns in the header row
        for i, column in enumerate(sheet.iter_cols(min_row=3, max_row=3, min_col=3), start=3):
            header_value = column[0].value  # Get the header value for the current column

            if header_value in column_names:
                date_str_1 = correct_date_1(header_value)
                date_str_2 = correct_date_2(header_value)

                if date_str_1 and date_str_2:
                    data_values_row1[date_str_1] = sheet.cell(row=row_num, column=i - 0).value
                    data_values_row2[date_str_2] = sheet.cell(row=row_num + 1, column=i - 0).value

                    if data_values_row1[date_str_1] is not None:

                        corrected_data_values_row1 = data_values_row1[date_str_1] * 1000 
                        corrected_data_values_row2 = data_values_row2[date_str_2] * 1000 

                        
                        val = (product_id, date_str_1, corrected_data_values_row1, 150, 0, 5) 
                        b_cursor.execute(check_data_query, val)
                        existing_data = b_cursor.fetchone() 

                        b_cursor.fetchall()

                        if existing_data:
                            print("Data already exists for", date_str_1, "and", product_id)
                            continue
                            
                        else: 
                            insert_query = "INSERT IGNORE INTO TradeData (product_id, date, raw_trade_quantity, destination_country_id_raw, origin_country_id, trade_quantity, raw_unit, unit, data_source_id, frequency) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                            b_cursor.execute(insert_query,(product_id, date_str_1, data_values_row1[date_str_1], 0 , 150, corrected_data_values_row1, 'kmt', 'mt', 5, 'monthly'))
                        
                        b_conn.commit()


    b_cursor.close()
    b_conn.close()

print(b_cursor.rowcount, "record(s) inserted.")

Data already exists for 2023-09-30 and 263
Data already exists for 2023-10-31 and 263
Data already exists for 2023-11-30 and 263
Data already exists for 2023-12-31 and 263
Data already exists for 2024-01-31 and 263
Data already exists for 2023-09-30 and 859
Data already exists for 2023-10-31 and 859
Data already exists for 2023-11-30 and 859
Data already exists for 2023-12-31 and 859
Data already exists for 2023-09-30 and 846
Data already exists for 2023-10-31 and 846
Data already exists for 2023-11-30 and 846
Data already exists for 2023-12-31 and 846
Data already exists for 2024-01-31 and 846
Data already exists for 2023-09-30 and 244
Data already exists for 2023-10-31 and 244
Data already exists for 2023-11-30 and 244
Data already exists for 2023-12-31 and 244
Data already exists for 2024-01-31 and 244
Data already exists for 2023-09-30 and 93
Data already exists for 2023-10-31 and 93
Data already exists for 2023-11-30 and 93
Data already exists for 2023-12-31 and 93
Data already ex